# Project 3 Part 3
- Michael Vincent
- 9/28

## Imports

In [1]:
# Imports
import numpy as np
import pandas as pd
import pymysql
from sqlalchemy import create_engine
from sqlalchemy_utils import create_database
from sqlalchemy.types import *

## Setup the SQL connection

In [2]:
# Setup the SQL connection

# Install the MySQLdb
pymysql.install_as_MySQLdb()

# Get the mysql username and password
with open('/home/michael/.secret/mysql.txt', 'r') as f:
    pw = f.read()
    pw = pw.replace('\n', '')
connection_str = 'mysql+pymysql://' + pw + '@localhost/movies.db'

# Construct the engine
engine = create_engine(connection_str)

# Create the database
try:
    create_database(connection_str)
except:
    print('Database Already Exists')

Database Already Exists


## Construct the title_genres and genres tables

In [3]:
# Load the basics data
basics = pd.read_csv('Data/title_basics.csv.gz')
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance"
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El Tango del Viudo y Su Espejo Deformante,0,2020,NaN,70,Drama
2,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,NaN,122,Drama
3,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005,NaN,100,"Comedy,Horror,Sci-Fi"
4,tt0096056,movie,Crime and Punishment,Crime and Punishment,0,2002,NaN,126,Drama


In [4]:
# Split the genres
basics['split_genres'] = basics['genres'].str.split(',')
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,split_genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance","[Comedy, Fantasy, Romance]"
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El Tango del Viudo y Su Espejo Deformante,0,2020,NaN,70,Drama,[Drama]
2,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,NaN,122,Drama,[Drama]
3,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005,NaN,100,"Comedy,Horror,Sci-Fi","[Comedy, Horror, Sci-Fi]"
4,tt0096056,movie,Crime and Punishment,Crime and Punishment,0,2002,NaN,126,Drama,[Drama]


In [5]:
# Separate the genres
exploded_genres = basics.explode('split_genres')
exploded_genres

# Get a list of unique genres
unique_genres = sorted(exploded_genres['split_genres'].unique())

In [6]:
# Create a title_genres table
title_genres = exploded_genres[['tconst', 'split_genres']].copy()
title_genres

,tconst,split_genres
0,tt0035423,Comedy
0,tt0035423,Fantasy
0,tt0035423,Romance
1,tt0062336,Drama
2,tt0069049,Drama
...,...,...
82469,tt9916190,Action
82469,tt9916190,Adventure
82469,tt9916190,Thriller
82470,tt9916362,Drama


In [7]:
# Make a genre mapper dictionary
genre_ints = range(len(unique_genres))
genre_map = dict(zip(unique_genres, genre_ints))
genre_map

{'Action': 0,
 'Adult': 1,
 'Adventure': 2,
 'Animation': 3,
 'Biography': 4,
 'Comedy': 5,
 'Crime': 6,
 'Drama': 7,
 'Family': 8,
 'Fantasy': 9,
 'Game-Show': 10,
 'History': 11,
 'Horror': 12,
 'Music': 13,
 'Musical': 14,
 'Mystery': 15,
 'News': 16,
 'Reality-TV': 17,
 'Romance': 18,
 'Sci-Fi': 19,
 'Short': 20,
 'Sport': 21,
 'Talk-Show': 22,
 'Thriller': 23,
 'War': 24,
 'Western': 25}

In [8]:
# Create a genre_id column in the title_genres data frame
title_genres['genre_id'] = title_genres['split_genres'].map(genre_map)
title_genres.drop(columns = 'split_genres', inplace = True)
title_genres.head()

,tconst,genre_id
0,tt0035423,5
0,tt0035423,9
0,tt0035423,18
1,tt0062336,7
2,tt0069049,7


In [9]:
# Convert the genre_map to a data frame
genres = pd.DataFrame({'genre_name': genre_map.keys(),
                       'genre_id': genre_map.values()})
genres.head()

,genre_name,genre_id
0,Action,0
1,Adult,1
2,Adventure,2
3,Animation,3
4,Biography,4


In [18]:
# Save the title_genres and genres tables

# Get the max string length of tconst and genre_name
key_len = title_genres['tconst'].fillna('').map(len).max()
genre_name_len = genres['genre_name'].fillna('').map(len).max()

# Make the schemas
title_genres_schema = {'tconst': String(key_len + 1),
                       'genere_id': Integer()}
genres_schema = {'genre_name': String(genre_name_len + 1), 
                 'genre_id': Integer()}

# Save the data frames to the database
title_genres.to_sql('title_genres', engine, dtype = title_genres_schema,
                    if_exists = 'replace', index = False)
genres.to_sql('genres', engine, dtype = genres_schema,
              if_exists = 'replace', index = False)

,tconst,genre_id
0,tt0035423,5
1,tt0035423,9
2,tt0035423,18
3,tt0062336,7
4,tt0069049,7
...,...,...
154144,tt9916190,0
154145,tt9916190,2
154146,tt9916190,23
154147,tt9916362,7


In [21]:
# Query the tables
display(pd.read_sql("SELECT * FROM title_genres;", engine),
        pd.read_sql("SELECT * FROM genres;", engine))

,tconst,genre_id
0,tt0035423,5
1,tt0035423,9
2,tt0035423,18
3,tt0062336,7
4,tt0069049,7
...,...,...
154144,tt9916190,0
154145,tt9916190,2
154146,tt9916190,23
154147,tt9916362,7


,genre_name,genre_id
0,Action,0
1,Adult,1
2,Adventure,2
3,Animation,3
4,Biography,4
5,Comedy,5
6,Crime,6
7,Drama,7
8,Family,8
9,Fantasy,9


## Construct the remaining tables for the database

In [11]:
# Load the necessary data frames
basics = pd.read_csv('Data/title_basics.csv.gz')
ratings = pd.read_csv('Data/title_ratings.csv.gz')
tmdb_data = pd.read_csv('Data/tmdb_results_combined.csv.gz')

# Drop the tables we don't want to keep
basics = basics[['tconst', 'primaryTitle', 'startYear',
                 'endYear', 'runtimeMinutes']]
ratings = ratings[['tconst', 'averageRating', 'numVotes']]
tmdb_data = tmdb_data[['imdb_id', 'budget', 'revenue', 'certification']]

In [12]:
# Get the lengths of the 
primaryTitle_len = basics['primaryTitle'].fillna('').map(len).max()
cert_len = tmdb_data['certification'].fillna('').map(len).max()

In [13]:
# Set the schemas
basics_schema = {'tconst': String(key_len + 1),
                 'primaryTitle': String(primaryTitle_len + 1),
                 'startYear': Integer(),
                 'endYear': Integer(),
                 'runtimeMinutes': Integer()}
ratings_schema = {'tconst': String(key_len + 1),
                  'averageRating': Float(),
                  'numVotes': Integer()}
tmdb_schema = {'imdb_id': String(key_len + 1),
               'budget': Float(),
               'revenue': Float(),
               'certification': String(cert_len + 1)}

In [14]:
# Save the data frames to the database
basics.to_sql('title_basics', engine, dtype = basics_schema,
              if_exists = 'replace', index = False)
ratings.to_sql('title_ratings', engine, dtype = ratings_schema,
               if_exists = 'replace', index = False)
tmdb_data.to_sql('tmdb_data', engine, dtype = tmdb_schema,
                 if_exists = 'replace', index = False)

2496

In [15]:
# Set the primary keys
engine.execute('ALTER TABLE title_basics ADD PRIMARY KEY (`tconst`);')
engine.execute('ALTER TABLE title_ratings ADD PRIMARY KEY (`tconst`);')
engine.execute('ALTER TABLE tmdb_data ADD PRIMARY KEY (`imdb_id`);')

In [32]:
# Query the tables
display(pd.read_sql("SELECT * FROM title_basics;", engine),
        pd.read_sql("SELECT * FROM title_ratings;", engine),
        pd.read_sql("SELECT * FROM tmdb_data;", engine))

,tconst,primaryTitle,startYear,endYear,runtimeMinutes
0,tt0035423,Kate & Leopold,2001,None,118
1,tt0062336,The Tango of the Widower and Its Distorting Mi...,2020,None,70
2,tt0069049,The Other Side of the Wind,2018,None,122
3,tt0088751,The Naked Monster,2005,None,100
4,tt0096056,Crime and Punishment,2002,None,126
...,...,...,...,...,...
82466,tt9914942,Life Without Sara Amat,2019,None,74
82467,tt9915872,The Last White Witch,2019,None,97
82468,tt9916170,The Rehearsal,2019,None,51
82469,tt9916190,Safeguard,2020,None,95


,tconst,averageRating,numVotes
0,tt0000001,5.7,1910
1,tt0000002,5.8,256
2,tt0000005,6.2,2527
3,tt0000006,5.1,173
4,tt0000007,5.4,789
...,...,...,...
476154,tt9916204,8.1,243
476155,tt9916348,8.5,17
476156,tt9916362,6.4,4863
476157,tt9916428,3.8,14


,imdb_id,budget,revenue,certification
0,tt0035423,48000000.0,76019000.0,PG-13
1,tt0113026,10000000.0,0.0,None
2,tt0113092,0.0,0.0,None
3,tt0114447,0.0,0.0,None
4,tt0116391,0.0,0.0,None
...,...,...,...,...
2491,tt7797790,0.0,0.0,None
2492,tt7802790,0.0,0.0,None
2493,tt8665056,0.0,0.0,None
2494,tt8795764,0.0,0.0,NR


In [33]:
# Show the tables
pd.read_sql("SHOW TABLES;", engine)

,Tables_in_movies.db
0,genres
1,title_basics
2,title_genres
3,title_ratings
4,tmdb_data
